In [1]:
import pandas as pd
import numpy as np
import glob
import tqdm
import string
import os

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

sw = stopwords.words('spanish')

#from thefuzz import fuzz
#from thefuzz import process

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import multiprocessing as mp
import istarmap  # import to apply patch

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

/Users/cevalenciam/anaconda3/envs/smartbuy/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
n = mp.cpu_count()

In [3]:
try:
   mp.set_start_method('spawn', force=True)
#   print("spawned")
except RuntimeError:
   pass

In [4]:
os.getcwd()

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/homologacion-smartbuy'

In [5]:
ruta = os.getcwd()

In [6]:
def marca_tottus(x):
    if x.find('marca: ')==-1:
        return('')
    elif x.find(';', x.find('marca: ')+1)==-1:
        return(x[x.find('marca: ')+7:])
    else:
        return(x[x.find('marca: ')+7: x.find(';', x.index('marca: ')+1)])

In [7]:
def formato_tottus(x):
    if x.find('formato: ')==-1:
        return('')
    elif x.find(';', x.find('formato: ')+1)==-1:
        return(x[x.find('formato: ')+9:])
    else:
        return(x[x.find('formato: ')+9: x.find(';', x.find('formato: ')+1)])

In [8]:
def format(x):
    if x[-2:]=='gr':
        return(x[:-1])
    elif x[-2:]=='lt':
        return(x[:-1])
    else:
        return(x)

In [9]:
def formato_lider(x):
    if x[x.rfind(',')+1:x.rfind(',')+2]!=' ':
        return(x[x.rfind(',', 0, x.rfind(','))+2:])
    else:
        return(x[x.rfind(',')+2:])

In [10]:
def remove_stopwords(lines, sw = sw):
    '''
    The purpose of this function is to remove stopwords from a given array of 
    lines.
    
    params:
        lines (Array / List) : The list of lines you want to remove the stopwords from
        sw (Set) : The set of stopwords you want to remove
        
    example:
        lines = remove_stopwords(lines = lines, sw = sw)
    '''
    
    res = []
    for line in lines:
        original = line
        line = [w for w in line if w not in sw]
        if len(line) < 1:
            line = original
        res.append(line)
    return res

In [92]:
def matching(id, product, empresa, collection_tottus, collection_lider, collection_jumbo):
    tottus = process.extractOne(product, collection_tottus, scorer=fuzz.ratio)
    if tottus==None:
        tottus = ('', np.nan)
    lider = process.extractOne(product, collection_lider, scorer=fuzz.ratio)
    if lider==None:
        lider = ('', np.nan)
    jumbo = process.extractOne(product, collection_jumbo, scorer=fuzz.ratio)
    if jumbo==None:
        jumbo = ('', np.nan)
    return([id, product, empresa, tottus[0], tottus[1], lider[0], lider[1], jumbo[0], jumbo[1]])

In [12]:
df_cat_smartbuy = pd.read_excel(ruta + "/Categorías_SmartBuy_v6.xlsx")

In [13]:
df_tottus = pd.read_excel(ruta[:-22] + "/web-scraper-tottus/Output/REPORTE_WS_TOTTUS.xlsx", converters={'internet_price':str})
print(df_tottus.shape[0])
df_tottus['id'] = 'T' + df_tottus['id'].fillna(0).astype(int).astype(str)
df_tottus['name'] = df_tottus['name'].astype(str).replace('nan', 'no info')
df_tottus['description'] = df_tottus['description'].astype(str).map(lambda x:x[x.find('>', x.find('h3'))+1:x.find('</h3>')] if x[0]=='<' else x).replace('nan', 'no info')
df_tottus['marca'] = df_tottus['attributes'].astype(str).str.lower().map(lambda x:marca_tottus(x)).replace('nan', 'no info')
df_tottus['formato'] = df_tottus['attributes'].astype(str).str.lower().map(lambda x:formato_tottus(x)).replace('nan', 'no info')
df_tottus['formato'] = df_tottus['formato'].map(lambda x:format(x))
df_tottus['image1'] = df_tottus['url_image'].astype(str).map(lambda x:x if x.find(';')==-1 else x[:x.find(';')]).replace('nan', 'no info')
df_tottus['image2'] = df_tottus['url_image'].astype(str).map(lambda x:'' if x.find(';')==-1 else x[x.find(';')+2:x.find(';', x.find(';')+1)]).replace('nan', 'no info')
df_tottus['image3'] = df_tottus['url_image'].astype(str).map(lambda x:'' if x.find(';', x.find(';')+1)==-1 else x[:x.find(';')]).replace('nan', 'no info')
df_tottus['internet_price'] = df_tottus['internet_price'].astype(str).str.replace('.', '').str.replace('nan', '0').astype(float)
df_tottus['empresa'] = 'Tottus'

#3 niveles
df_tottus = pd.merge(df_tottus, 
                df_cat_smartbuy[df_cat_smartbuy['Categoría nivel 3.1'].notnull()][['Categoría nivel 1.2', 'Categoría nivel 2.2', 'Categoría nivel 3.1', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']]\
                .rename(columns={'Categoría nivel 1.2': 'first_category',
                                'Categoría nivel 2.2': 'second_category',
                                'Categoría nivel 3.1': 'third_category'}).drop_duplicates(), 
                on=['first_category', 'second_category', 'third_category'], 
                how='left').drop_duplicates()

#2 niveles
df_tottus = pd.merge(df_tottus, 
                df_cat_smartbuy[(df_cat_smartbuy['Categoría nivel 3.1'].isnull()) &
                                (df_cat_smartbuy['Categoría nivel 2.2'].notnull())][['Categoría nivel 1.2', 'Categoría nivel 2.2', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']]\
                .rename(columns={'Categoría nivel 1.2': 'first_category',
                                'Categoría nivel 2.2': 'second_category'}).drop_duplicates(), 
                on=['first_category', 'second_category'], 
                how='left').drop_duplicates()

#creando campos 
df_tottus['COD_CAT_N1_SMT'] = np.where(df_tottus['COD_CAT_N1_SMT_x'].isnull(), df_tottus['COD_CAT_N1_SMT_y'], df_tottus['COD_CAT_N1_SMT_x'])
df_tottus['Categoría nivel 1 - Smartbuy'] = np.where(df_tottus['Categoría nivel 1 - Smartbuy_x'].isnull(), df_tottus['Categoría nivel 1 - Smartbuy_y'], df_tottus['Categoría nivel 1 - Smartbuy_x'])
df_tottus['COD_CAT_N2_SMT'] = np.where(df_tottus['COD_CAT_N2_SMT_x'].isnull(), df_tottus['COD_CAT_N2_SMT_y'], df_tottus['COD_CAT_N2_SMT_x'])
df_tottus['Categoría nivel 2 - Smartbuy'] = np.where(df_tottus['Categoría nivel 2 - Smartbuy_x'].isnull(), df_tottus['Categoría nivel 2 - Smartbuy_y'], df_tottus['Categoría nivel 2 - Smartbuy_x'])

#base final
df_tottus = df_tottus[df_tottus['COD_CAT_N1_SMT'].notnull()].drop(['COD_CAT_N1_SMT_x',
                                                                'Categoría nivel 1 - Smartbuy_x', 'COD_CAT_N2_SMT_x',
                                                                'Categoría nivel 2 - Smartbuy_x', 'COD_CAT_N1_SMT_y',
                                                                'Categoría nivel 1 - Smartbuy_y', 'COD_CAT_N2_SMT_y',
                                                                'Categoría nivel 2 - Smartbuy_y'], axis=1).drop_duplicates()

print(df_tottus.shape[0])

6330
695


In [14]:
filenames = glob.glob(ruta[:-22] + "/web-scraper-lider/Output/REPORTE_WS_*")
df_lider = pd.concat(pd.read_excel(file, converters={'internet_price':str}) for file in filenames).reset_index(drop=True)
print(df_lider.shape[0])
df_lider['id'] = 'L' + df_lider['id'].fillna('item 0').map(lambda x:x[5:])
df_lider['name'] = df_lider['name'].astype(str).replace('nan', 'no info')
df_lider['description'] = df_lider['description'].astype(str).replace('nan', 'no info')
df_lider['marca'] = df_lider['attributes'].astype(str).str.lower().str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u').replace('nan', 'no info')
df_lider['formato'] = df_lider['name'].astype(str).str.lower().map(lambda x:formato_lider(x)).replace('nan', 'no info')
df_lider['formato'] = df_lider['formato'].map(lambda x:format(x))
df_lider['image1'] = df_lider['url_image'].astype(str).map(lambda x:x if x.find(';')==-1 else x[:x.find(';')]).replace('nan', 'no info')
df_lider['image2'] = df_lider['url_image'].astype(str).map(lambda x:'' if x.find(';')==-1 else x[x.find(';')+2:x.find(';', x.find(';')+1)]).replace('nan', 'no info')
df_lider['image3'] = df_lider['url_image'].astype(str).map(lambda x:'' if x.find(';', x.find(';')+1)==-1 else x[:x.find(';')]).replace('nan', 'no info')
df_lider['internet_price'] = np.where(df_lider['internet_price'].astype(str).map(lambda x:x[0].isdigit()), df_lider['normal_price'], df_lider['internet_price'])
df_lider['internet_price'] = df_lider['internet_price'].astype(str).map(lambda x:x[x.find('$')+1:]).str.replace('.', '').str.replace('nan', '0').astype(float)
df_lider['empresa'] = 'Lider'

#2 categorias
df_lider = pd.merge(df_lider, 
                df_cat_smartbuy[['Categoría nivel 1', 'Categoría nivel 2', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']]\
                .rename(columns={'Categoría nivel 1': 'first_category',
                                'Categoría nivel 2': 'second_category'}).drop_duplicates(), 
                on=['first_category', 'second_category'], 
                how='inner').drop_duplicates()

print(df_lider.shape[0])

57380
23288


In [15]:
filenames = glob.glob(ruta[:-22] + "/web-scraper-jumbo/Output/REPORTE_WS_*")
df_jumbo = pd.concat(pd.read_excel(file, converters={'internet_price':str}) for file in filenames).reset_index(drop=True)
print(df_jumbo.shape[0])
df_jumbo['id'] = 'J' + df_jumbo['id'].fillna('Código: 0').map(lambda x:x[8:])
df_jumbo['name'] = df_jumbo['name'].astype(str).replace('nan', 'no info')
df_jumbo['description'] = df_jumbo['description'].astype(str).replace('nan', 'no info')
df_jumbo['marca'] = df_jumbo['attributes'].astype(str).str.lower().str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u').replace('nan', 'no info')
df_jumbo['formato'] = df_jumbo['name'].astype(str).str.lower().map(lambda x:x[x[:x.rfind(" ")].rfind(" ")+1:]).replace('nan', 'no info')
df_jumbo['formato'] = df_jumbo['formato'].map(lambda x:format(x))
df_jumbo['image1'] = df_jumbo['url_image'].astype(str).map(lambda x:x if x.find(';')==-1 else x[:x.find(';')]).replace('nan', 'no info')
df_jumbo['image2'] = df_jumbo['url_image'].astype(str).map(lambda x:'' if x.find(';')==-1 else x[x.find(';')+2:x.find(';', x.find(';')+1)]).replace('nan', 'no info')
df_jumbo['image3'] = df_jumbo['url_image'].astype(str).map(lambda x:'' if x.find(';', x.find(';')+1)==-1 else x[:x.find(';')]).replace('nan', 'no info')
df_jumbo['internet_price'] = df_jumbo['internet_price'].astype(str).map(lambda x:x[x.find('$')+1:]).str.replace('.', '').str.replace('nan', '0').astype(float)
df_jumbo['empresa'] = 'Jumbo'

#3 niveles
df_jumbo = pd.merge(df_jumbo, 
                df_cat_smartbuy[df_cat_smartbuy['Categoría nivel 3'].notnull()][['Categoría nivel 1.1', 'Categoría nivel 2.1', 'Categoría nivel 3', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']]\
                .rename(columns={'Categoría nivel 1.1': 'first_category',
                                'Categoría nivel 2.1': 'second_category',
                                'Categoría nivel 3': 'third_category'}).drop_duplicates(), 
                on=['first_category', 'second_category', 'third_category'], 
                how='left').drop_duplicates()

#2 niveles
df_jumbo = pd.merge(df_jumbo, 
                df_cat_smartbuy[(df_cat_smartbuy['Categoría nivel 3'].isnull()) &
                                (df_cat_smartbuy['Categoría nivel 2.1'].notnull())][['Categoría nivel 1.1', 'Categoría nivel 2.1', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']]\
                .rename(columns={'Categoría nivel 1.1': 'first_category',
                                'Categoría nivel 2.1': 'second_category'}).drop_duplicates(), 
                on=['first_category', 'second_category'], 
                how='left').drop_duplicates()

#creando campos 
df_jumbo['COD_CAT_N1_SMT'] = np.where(df_jumbo['COD_CAT_N1_SMT_x'].isnull(), df_jumbo['COD_CAT_N1_SMT_y'], df_jumbo['COD_CAT_N1_SMT_x'])
df_jumbo['Categoría nivel 1 - Smartbuy'] = np.where(df_jumbo['Categoría nivel 1 - Smartbuy_x'].isnull(), df_jumbo['Categoría nivel 1 - Smartbuy_y'], df_jumbo['Categoría nivel 1 - Smartbuy_x'])
df_jumbo['COD_CAT_N2_SMT'] = np.where(df_jumbo['COD_CAT_N2_SMT_x'].isnull(), df_jumbo['COD_CAT_N2_SMT_y'], df_jumbo['COD_CAT_N2_SMT_x'])
df_jumbo['Categoría nivel 2 - Smartbuy'] = np.where(df_jumbo['Categoría nivel 2 - Smartbuy_x'].isnull(), df_jumbo['Categoría nivel 2 - Smartbuy_y'], df_jumbo['Categoría nivel 2 - Smartbuy_x'])

#base final
df_jumbo = df_jumbo[df_jumbo['COD_CAT_N1_SMT'].notnull()].drop(['COD_CAT_N1_SMT_x',
                                                                'Categoría nivel 1 - Smartbuy_x', 'COD_CAT_N2_SMT_x',
                                                                'Categoría nivel 2 - Smartbuy_x', 'COD_CAT_N1_SMT_y',
                                                                'Categoría nivel 1 - Smartbuy_y', 'COD_CAT_N2_SMT_y',
                                                                'Categoría nivel 2 - Smartbuy_y'], axis=1).drop_duplicates()

print(df_jumbo.shape[0])

30573
14364


In [163]:
vars_productos = ['id', 'name', 'description', 'first_category', 'second_category', 'third_category', 'marca', 'formato', 'internet_price', 'image1', 'image2', 'image3', 'empresa', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']

In [18]:
df_prod_1 = df_prod[vars_productos].reset_index(drop=True)

In [19]:
print("Total: ",df_prod_1.shape)
df_prod_1 = df_prod_1.drop_duplicates()
print("Total sin duplicados: ",df_prod_1.shape)

Total:  (38347, 17)
Total sin duplicados:  (38299, 17)


In [414]:
df_prod_2 = df_prod_1.groupby(['id', 'name', 'marca', 'formato', 'empresa'], as_index=False, dropna=False).size()
df_prod_2.head()

,id,name,marca,formato,empresa,size
0,J1000589,Pan de molde blanco 580 g,pan pierre,580 g,Jumbo,1
1,J1000590,Pan de molde integral 580 g,pan pierre,580 g,Jumbo,1
2,J1000623,Agua Mineral Jahuel Gasificada 330 cc,jahuel,330 cc,Jumbo,2
3,J1000624,Agua Mineral Jahuel Sin Gas de 330 cc,jahuel,330 cc,Jumbo,2
4,J1000627,Agua Mineral Jahuel Gasificada 1.6 L,jahuel,1.6 l,Jumbo,2


In [22]:
df = df_prod_1[~((df_prod_1['id'].isin(df_prod_2[df_prod_2['size']>1]['id'].unique())) & (df_prod_1['third_category'].fillna('sin info').str.lower().str.contains('ver todo')))]

In [23]:
vars_productos_2 = ['id', 'name', 'description', 'marca', 'formato', 'internet_price', 'image1', 'image2', 'image3', 'empresa', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']

In [24]:
df = df[vars_productos_2].reset_index(drop=True)

In [25]:
print("Total: ",df.shape)
df = df.drop_duplicates()
print("Total sin duplicados: ",df.shape)

Total:  (23269, 14)
Total sin duplicados:  (22332, 14)


In [33]:
df['product'] = df['name'].astype(str) + ' ' +\
                df['marca'].astype(str) + ' ' +\
                df['formato'].astype(str)

In [35]:
lines = df['product'].tolist()

# remove new lines
lines = [line.rstrip('\n') for line in lines]

# make all characters lower
lines = [line.lower() for line in lines]

# remove punctuations from each line
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]

# tokenize
lines = [word_tokenize(line) for line in lines]
    
filtered_lines = remove_stopwords(lines = lines, sw = sw)

filtered_lines = [TreebankWordDetokenizer().detokenize(line) for line in filtered_lines]

In [36]:
df['product'] = filtered_lines

In [824]:
df[df['id']=='L293383']

,id,name,description,marca,formato,internet_price,image1,image2,image3,empresa,COD_CAT_N1_SMT,Categoría nivel 1 - Smartbuy,COD_CAT_N2_SMT,Categoría nivel 2 - Smartbuy,product
10782,L293383,"Aceite De Maravilla Botella Botella, 1 L","Aceite Vegetal Maravilla Chef, 1lt",chef,1 l,3790.0,https://images.lider.cl/wmtcl?source=url[file:...,,,Lider,D_001,Despensa,AYA_001,Aceites y Aderezos,aceite maravilla botella botella 1 l chef 1 l


In [833]:
categorias_smt = df[df['Categoría nivel 2 - Smartbuy'].str.lower().str.contains('aceites')][['COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy']].drop_duplicates().values.tolist()

In [830]:
categorias_smt = categorias_smt[-2:-1]
categorias_smt

[]

In [835]:
results = []

i = 1

df_res = pd.DataFrame()

for cat1, cat_1, cat2, cat_2 in categorias_smt:

    print(cat_1, ' ', cat_2)
    print(i, ' de ', len(categorias_smt))

    collection_tottus = df[(df['empresa']=='Tottus') & (df['COD_CAT_N1_SMT']==cat1) & (df['COD_CAT_N2_SMT']==cat2)]['product'].values
    collection_lider = df[(df['empresa']=='Lider') & (df['COD_CAT_N1_SMT']==cat1) & (df['COD_CAT_N2_SMT']==cat2)]['product'].values
    collection_jumbo = df[(df['empresa']=='Jumbo') & (df['COD_CAT_N1_SMT']==cat1) & (df['COD_CAT_N2_SMT']==cat2)]['product'].values

    products = df[(df['COD_CAT_N1_SMT']==cat1) & (df['COD_CAT_N2_SMT']==cat2)][['id', 'product', 'empresa']].drop_duplicates().values

    for id, product, empresa in tqdm.tqdm(products):
        results.append(matching(id, product, empresa, collection_tottus, collection_lider, collection_jumbo))


    df_temp = pd.DataFrame(results, columns=['id', 'product', 'empresa', 'name_tottus', 'score_tottus', 'name_lider', 'score_lider', 'name_jumbo', 'score_jumbo'])

    df_res = df_res.append(df_temp)

    i = i+1

Despensa   Aceites y Aderezos
1  de  1


100%|██████████| 783/783 [01:35<00:00,  8.21it/s]


In [219]:
df_matching = pd.read_excel('df_matching.xlsx')

In [221]:
df_matching['product'] = df_matching['name'].astype(str) + ' ' +\
                df_matching['marca'].astype(str) + ' ' +\
                df_matching['formato'].astype(str)

In [222]:
lines = df_matching['product'].tolist()

# remove new lines
lines = [line.rstrip('\n') for line in lines]

# make all characters lower
lines = [line.lower() for line in lines]

# remove punctuations from each line
lines = [line.translate(str.maketrans('', '', string.punctuation)) for line in lines]

# tokenize
lines = [word_tokenize(line) for line in lines]
    
filtered_lines = remove_stopwords(lines = lines, sw = sw)

filtered_lines = [TreebankWordDetokenizer().detokenize(line) for line in filtered_lines]

In [223]:
df_matching['product'] = filtered_lines

In [224]:
df_res = df_matching[df_res.columns].drop_duplicates()

In [518]:
df_res.head(3)

,id,product,empresa,name_tottus,score_tottus,name_lider,score_lider,name_jumbo,score_jumbo
0,T110606885,aceite vegetal tottus 900 cc nan 900 ml,Tottus,aceite vegetal tottus 900 cc 900 ml,100.0,aceite vegetal 900 ml romana 900 ml,72,aceite vegetal 900 ml natura 900 ml,72.0
1,T110615978,aceite vegetal belmont 1 l,Tottus,aceite vegetal belmont 1 l,100.0,aceite vegetal botella 1 l belmont 1 l,81,aceite vegetal 1 l belmont 1 l,93.0
4,T110607018,aceite oliva extra virgen 500 ml tottus 500 ml,Tottus,aceite oliva extra virgen 500 ml tottus 500 ml,100.0,aceite oliva extra virgen botella 500 ml olave...,81,aceite oliva extra virgen 500 ml trattoria 500 ml,89.0


In [836]:
df_res_0 = pd.merge(df_res, df[['id', 'product', 'empresa', 'name', 'description', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy', 'marca', 'formato', 'internet_price', 'image1', 'image2', 'image3']].drop_duplicates(), how='left', on=['id', 'product', 'empresa']).reset_index(drop=True)

In [837]:
df_res_0 = df_res_0[['id', 'name', 'description', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy', 'marca', 'formato', 'internet_price', 'image1', 'image2', 'image3', 'product', 'empresa', 'name_tottus', 'score_tottus', 'name_lider', 'score_lider', 'name_jumbo', 'score_jumbo']].drop_duplicates()

In [838]:
df_res[df_res['id']=='T110621059']

,id,product,empresa,name_tottus,score_tottus,name_lider,score_lider,name_jumbo,score_jumbo
5,T110621059,aceite maravilla 1 lt chef 1 l,Tottus,aceite maravilla 1 lt chef 1 l,100,aceite maiz botella 1 l chef 1 l,84,aceite maravilla 1 l chef 1 l,98


In [839]:
df_res_0.shape

(792, 21)

In [788]:
df_res_1 = pd.merge(df_res_0, df[df['empresa']=='Tottus'][['product', 'marca', 'formato', 'internet_price']].rename(columns={'product': 'name_tottus',
                                                                                                                                'marca': 'marca_tottus',
                                                                                                                                'formato': 'formato_tottus',
                                                                                                                                'internet_price': 'precio_tottus'}), how='left', on='name_tottus')

In [789]:
df_res_1.shape

(26922, 24)

In [790]:
df_res_2 = pd.merge(df_res_1, df[df['empresa']=='Lider'][['product', 'marca', 'formato', 'internet_price']].rename(columns={'product': 'name_lider',
                                                                                                                                'marca': 'marca_lider',
                                                                                                                                'formato': 'formato_lider',
                                                                                                                                'internet_price': 'precio_lider'}), how='left', on='name_lider')

In [791]:
df_res_2.shape

(32503, 27)

In [792]:
df_res_3 = pd.merge(df_res_2, df[df['empresa']=='Jumbo'][['product', 'marca', 'formato', 'internet_price']].rename(columns={'product': 'name_jumbo',
                                                                                                                                'marca': 'marca_jumbo',
                                                                                                                                'formato': 'formato_jumbo',
                                                                                                                                'internet_price': 'precio_jumbo'}), how='left', on='name_jumbo')

In [793]:
df_res_3.shape

(32870, 30)

In [794]:
df_res_4 = df_res_3[['id', 'name', 'description', 'COD_CAT_N1_SMT', 'Categoría nivel 1 - Smartbuy', 'COD_CAT_N2_SMT', 'Categoría nivel 2 - Smartbuy', 'marca', 'formato', 'internet_price', 'image1', 'image2', 'image3', 'empresa', 'name_tottus', 'score_tottus', 'marca_tottus', 'formato_tottus', 'precio_tottus', 'name_lider', 'score_lider', 'marca_lider', 'formato_lider', 'precio_lider', 'name_jumbo', 'score_jumbo', 'marca_jumbo', 'formato_jumbo', 'precio_jumbo']].drop_duplicates()

In [795]:
df_res_4.shape

(24525, 29)

In [796]:
df_res_4['score_fuzz_tottus'] = df_res_4['score_tottus'].copy()
df_res_4['score_fuzz_lider'] = df_res_4['score_lider'].copy()
df_res_4['score_fuzz_jumbo'] = df_res_4['score_jumbo'].copy()

In [797]:
penalidad_formato = 5
penalidad_marca = 2
umbral_precio = 0.5
penalidad_precio = 5

df_res_4['penalidad_formato_tottus'] = np.where(df_res_4['formato'] == df_res_4['formato_tottus'], 0, penalidad_formato)
df_res_4['penalidad_formato_lider'] = np.where(df_res_4['formato'] == df_res_4['formato_lider'], 0, penalidad_formato)
df_res_4['penalidad_formato_jumbo'] = np.where(df_res_4['formato'] == df_res_4['formato_jumbo'], 0, penalidad_formato)

df_res_4['penalidad_marca_tottus'] = np.where(df_res_4['marca'] == df_res_4['marca_tottus'], 0, penalidad_marca)
df_res_4['penalidad_marca_lider'] = np.where(df_res_4['marca'] == df_res_4['marca_lider'], 0, penalidad_marca)
df_res_4['penalidad_marca_jumbo'] = np.where(df_res_4['marca'] == df_res_4['marca_jumbo'], 0, penalidad_marca)

df_res_4['penalidad_precio_tottus'] = np.where((abs((df_res_4['precio_tottus']-df_res_4['internet_price'])/df_res_4['internet_price'])<=umbral_precio), 0, penalidad_precio)
df_res_4['penalidad_precio_lider'] = np.where((abs((df_res_4['precio_tottus']-df_res_4['internet_price'])/df_res_4['internet_price'])<=umbral_precio), 0, penalidad_precio)
df_res_4['penalidad_precio_jumbo'] = np.where((abs((df_res_4['precio_tottus']-df_res_4['internet_price'])/df_res_4['internet_price'])<=umbral_precio), 0, penalidad_precio)

df_res_4['score_tottus'] = df_res_4['score_fuzz_tottus'] - df_res_4['penalidad_formato_tottus'] - df_res_4['penalidad_marca_tottus'] - df_res_4['penalidad_precio_tottus']
df_res_4['score_lider'] = df_res_4['score_fuzz_lider'] - df_res_4['penalidad_formato_lider'] - df_res_4['penalidad_marca_lider'] - df_res_4['penalidad_precio_lider']
df_res_4['score_jumbo'] = df_res_4['score_fuzz_jumbo'] - df_res_4['penalidad_formato_jumbo'] - df_res_4['penalidad_marca_jumbo'] - df_res_4['penalidad_precio_jumbo']

In [798]:
umbral_score = 60

In [799]:
# Tottus
df_res_4['name_tottus_inicial'] = df_res_4['name_tottus'].copy()
df_res_4['score_tottus_inicial'] = df_res_4['score_tottus'].copy()
df_res_4['marca_tottus_inicial'] = df_res_4['marca_tottus'].copy()
df_res_4['formato_tottus_inicial'] = df_res_4['formato_tottus'].copy()
df_res_4['precio_tottus_inicial'] = df_res_4['precio_tottus'].copy()

df_res_4['name_tottus'] = np.where((df_res_4['score_tottus_inicial']>=umbral_score), df_res_4['name_tottus'], '')
df_res_4['score_tottus'] = np.where((df_res_4['score_tottus_inicial']>=umbral_score), df_res_4['score_tottus'], np.nan)
df_res_4['marca_tottus'] = np.where((df_res_4['score_tottus_inicial']>=umbral_score), df_res_4['marca_tottus'], '')
df_res_4['formato_tottus'] = np.where((df_res_4['score_tottus_inicial']>=umbral_score), df_res_4['formato_tottus'], '')
df_res_4['precio_tottus'] = np.where((df_res_4['score_tottus_inicial']>=umbral_score), df_res_4['precio_tottus'], np.nan)

In [800]:
# Lider
df_res_4['name_lider_inicial'] = df_res_4['name_lider'].copy()
df_res_4['score_lider_inicial'] = df_res_4['score_lider'].copy()
df_res_4['marca_lider_inicial'] = df_res_4['marca_lider'].copy()
df_res_4['formato_lider_inicial'] = df_res_4['formato_lider'].copy()
df_res_4['precio_lider_inicial'] = df_res_4['precio_lider'].copy()

df_res_4['name_lider'] = np.where((df_res_4['score_lider_inicial']>=umbral_score), df_res_4['name_lider'], '')
df_res_4['score_lider'] = np.where((df_res_4['score_lider_inicial']>=umbral_score), df_res_4['score_lider'], np.nan)
df_res_4['marca_lider'] = np.where((df_res_4['score_lider_inicial']>=umbral_score), df_res_4['marca_lider'], '')
df_res_4['formato_lider'] = np.where((df_res_4['score_lider_inicial']>=umbral_score), df_res_4['formato_lider'], '')
df_res_4['precio_lider'] = np.where((df_res_4['score_lider_inicial']>=umbral_score), df_res_4['precio_lider'], np.nan)

In [801]:
# Jumbo
df_res_4['name_jumbo_inicial'] = df_res_4['name_jumbo'].copy()
df_res_4['score_jumbo_inicial'] = df_res_4['score_jumbo'].copy()
df_res_4['marca_jumbo_inicial'] = df_res_4['marca_jumbo'].copy()
df_res_4['formato_jumbo_inicial'] = df_res_4['formato_jumbo'].copy()
df_res_4['precio_jumbo_inicial'] = df_res_4['precio_jumbo'].copy()

df_res_4['name_jumbo'] = np.where((df_res_4['score_jumbo_inicial']>=umbral_score), df_res_4['name_jumbo'], '')
df_res_4['score_jumbo'] = np.where((df_res_4['score_jumbo_inicial']>=umbral_score), df_res_4['score_jumbo'], np.nan)
df_res_4['marca_jumbo'] = np.where((df_res_4['score_jumbo_inicial']>=umbral_score), df_res_4['marca_jumbo'], '')
df_res_4['formato_jumbo'] = np.where((df_res_4['score_jumbo_inicial']>=umbral_score), df_res_4['formato_jumbo'], '')
df_res_4['precio_jumbo'] = np.where((df_res_4['score_jumbo_inicial']>=umbral_score), df_res_4['precio_jumbo'], np.nan)


In [802]:
df_res_4 = df_res_4.drop_duplicates()

In [803]:
df_res_4 = df_res_4.rename(columns={'COD_CAT_N1_SMT': 'cod_cat_n1_smt',
                                    'Categoría nivel 1 - Smartbuy': 'categoria_nivel_1',
                                    'COD_CAT_N2_SMT': 'cod_cat_n2_smt',
                                    'Categoría nivel 2 - Smartbuy': 'categoria_nivel_2',
                                    'internet_price': 'precio'})

In [804]:
actual_date = str(datetime.now())[0:10]

In [805]:
df_res_4.to_excel(ruta + f'/df_matching_total.xlsx'.format(actual_date), index=0)

In [806]:
df_res_4.shape

(24525, 56)

In [807]:
df_res_5 = df_res_4[['id', 'name', 'description', 'cod_cat_n1_smt', 'categoria_nivel_1', 'cod_cat_n2_smt', 'categoria_nivel_2', 'marca', 'formato', 'precio', 'image1', 'image2', 'image3', 'empresa', 'name_tottus', 'score_tottus', 'marca_tottus', 'formato_tottus', 'precio_tottus', 'name_lider', 'score_lider', 'marca_lider', 'formato_lider', 'precio_lider', 'name_jumbo', 'score_jumbo', 'marca_jumbo', 'formato_jumbo', 'precio_jumbo']].drop_duplicates()
df_res_5.shape

(23746, 29)

In [808]:
cols_key = ['id', 'cod_cat_n1_smt', 'cod_cat_n2_smt']
cols_tottus = ['name_tottus', 'score_tottus', 'marca_tottus', 'formato_tottus', 'precio_tottus']
cols_lider = ['name_lider', 'score_lider', 'marca_lider', 'formato_lider', 'precio_lider']
cols_jumbo = ['name_jumbo', 'score_jumbo', 'marca_jumbo', 'formato_jumbo', 'precio_jumbo']

In [809]:
# Tottus
df_det_tottus = df_res_5[cols_key + cols_tottus]
df_det_tottus.sort_values(['score_tottus'], ascending=[False], inplace=True)
df_det_tottus['rank'] = 1
df_det_tottus['rank'] = df_det_tottus.groupby(cols_key, dropna=False)['rank'].cumsum()
df_det_tottus = df_det_tottus[df_det_tottus['rank']==1]
del df_det_tottus['rank']
df_det_tottus.shape, df_det_tottus[cols_key].drop_duplicates().shape

((22311, 8), (22311, 3))

In [810]:
# Lider
df_det_lider = df_res_5[cols_key + cols_lider]
df_det_lider.sort_values(['score_lider'], ascending=[False], inplace=True)
df_det_lider['rank'] = 1
df_det_lider['rank'] = df_det_lider.groupby(cols_key, dropna=False)['rank'].cumsum()
df_det_lider = df_det_lider[df_det_lider['rank']==1]
del df_det_lider['rank']
df_det_lider.shape, df_det_lider[cols_key].drop_duplicates().shape

((22311, 8), (22311, 3))

In [811]:
# Jumbo
df_det_jumbo = df_res_5[cols_key + cols_jumbo]
df_det_jumbo.sort_values(['score_jumbo'], ascending=[False], inplace=True)
df_det_jumbo['rank'] = 1
df_det_jumbo['rank'] = df_det_jumbo.groupby(cols_key, dropna=False)['rank'].cumsum()
df_det_jumbo = df_det_jumbo[df_det_jumbo['rank']==1]
del df_det_jumbo['rank']
df_det_jumbo.shape, df_det_jumbo[cols_key].drop_duplicates().shape

((22311, 8), (22311, 3))

In [812]:
df_res_6 = df_res_5.drop(cols_tottus, axis=1).drop(cols_lider, axis=1).drop(cols_jumbo, axis=1).drop_duplicates()
df_res_6.shape

(22313, 14)

In [813]:
df_res_6.sort_values(['image1'], ascending=[True], inplace=True)
df_res_6['rank'] = 1
df_res_6['rank'] = df_res_6.groupby(cols_key, dropna=False)['rank'].cumsum()
df_res_6 = df_res_6[df_res_6['rank']==1]
del df_res_6['rank']
df_res_6.shape, df_res_6[cols_key].drop_duplicates().shape

((22311, 14), (22311, 3))

In [814]:
df_res_7 = pd.merge(df_res_6, df_det_tottus, on=cols_key, how='left')
df_res_8 = pd.merge(df_res_7, df_det_lider, on=cols_key, how='left')
df_res_9 = pd.merge(df_res_8, df_det_jumbo, on=cols_key, how='left')

In [843]:
ruta

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/homologacion-smartbuy'

In [845]:
df_mmpp = pd.read_excel(ruta + '/../data/mmpp.xlsx')

In [846]:
lista_mmpp = df_mmpp['mmpp'].values.tolist()

In [847]:
lista_mmpp

['lider', 'cuisine & co', 'tottus']

In [848]:
df_res_9['flag_mmpp'] = 0

In [849]:
for mmpp in lista_mmpp:
    df_res_9['flag_mmpp'] = np.where(df_res_9['marca'].str.contains(mmpp), 1, df_res_9['flag_mmpp'])

In [850]:
df_res_9['flag_mmpp'] = np.where(df_res_9['marca'].isnull(), 0, df_res_9['flag_mmpp'])

In [852]:
df_final = df_res_9.copy()

In [853]:
print(df_final.shape)

print(df_final[['id', 'name', 'empresa', 'cod_cat_n1_smt', 'cod_cat_n2_smt']].drop_duplicates().shape)

(22311, 33)
(22311, 5)


In [855]:
df_final[['id', 'name', 'description', 'cod_cat_n1_smt', 'categoria_nivel_1', 'cod_cat_n2_smt', 'categoria_nivel_2', 'marca', 'formato', 'precio', 'image1', 'image2', 'image3', 'empresa', 'name_tottus', 'score_tottus', 'marca_tottus', 'formato_tottus', 'precio_tottus', 'name_lider', 'score_lider', 'marca_lider', 'formato_lider', 'precio_lider', 'name_jumbo', 'score_jumbo', 'marca_jumbo', 'formato_jumbo', 'precio_jumbo', 'flag_mmpp']].drop_duplicates().to_excel(ruta + f'/df_matching.xlsx'.format(actual_date), index=0)